# Learning how to map, localize and navigate wheeled robots with ROS

This notebook is additional material for the workshop created and provided for free by Román Navarro García. You can copy and distribute this notebook as long as you provide copy of this paragraph with it.


## Why this course?

Either you are starting with ROS or you already know something about how it works, there are some topics considered as basic ones that we should learn and improve continuosly. Amongst these topics, mapping, localization and navigation would be on top (or almost) of them.
The goal of this course is to go through all the steps to create a map, load and localize a robot on it, and navigate through it by using the standard or most extended packages provided by ROS.

## The robot

![Summit XL Steel](images/SUMMIT_XL_STEEL_101.jpg)

For this course we are going to use the [Summit XL Steel](https://www.robotnik.eu/mobile-robots/summit-xl-steel/) mobile robot. It is a four motors omni-directional robot with two 2D lidars to provide 360 degrees of coverage, two pan-tilt-zoom cameras, one (or two) rgbd camera, one IMU and a gps.

## The packages 

This tutorial is intended to work for ROS Kinetic distro, though it should be compatible with newer versions. 
All the packages used are available in the official ROS distro and in the source repositories:

* [robotnik_msgs](https://github.com/RobotnikAutomation/robotnik_msgs)
* [robotnik_sensors](https://github.com/RobotnikAutomation/robotnik_sensors)
* [summit_xl_common](https://github.com/RobotnikAutomation/summit_xl_common)
* [summit_xl_sim](https://github.com/RobotnikAutomation/summit_xl_sim)

## Contents

1. How to create a map of an indoor environment
  * Running the robot
  * Running gmapping
  * Moving the robot through the environment
  * Saving the map
2. How to use the map to localize the robot
  * Running map_server
  * Running amcl
3. How to use the map and localization to make the robot move autonomously on the environment while avoiding obstacles
  * Running move_base
  * Rviz Navigation Display
  * move_base config files

## 1. How to create a map of an indoor environment

In this section we will create and save a map by using the Summit XL Steel robot. To do that we will use the packages [gmapping](http://wiki.ros.org/gmapping) and [map_server](http://wiki.ros.org/map_server)

* **gmapping** provides laser-based SLAM (Simultaneous Localization and Mapping). You can create a 2D occupancy grid map from laser and pose data.

* **map_server** offers map data as a ROS Service. It also provides the **map_saver** command-line utility, which allows dynamically generated maps to be saved to file.

### 1.1 Running the robot

To run the robot and the demo environment you can use the following command:


In [ ]:
roslaunch summit_xl_sim_bringup summit_xls_complete.launch

You can also look the launch file directly in the **Simulations panel**. 

After running the robot you should see something like this in Gazebo:

![SummitXLS Gazebo](images/rdc_gazebo1.png)

And something like this in RVIZ:

![SummitXLS RVIZ](images/rdc_rviz1.png)

This launch file is configured to allow the execution of multiple robots (up to 3) and multiple options like running navigation, localization, etc. on startup. 

By default, just the robot controller and sensors are launched. To simulate the omni-directional movement (x, y, thetha) , the robot uses the [Gazebo Planar Move Plugin](http://gazebosim.org/tutorials?tut=ros_gzplugins#PlanarMovePlugin). The rest of plugins for the sensors are also standard Gazebo plugins.


### 1.2 Running gmapping

To work with gmapping we need:
 * 2D Laserscan
 * Robot's odometry
 * Tfs

To start gmapping we can run the following command in a terminal:

In [ ]:
ROS_NAMESPACE=summit_xl_a roslaunch summit_xl_localization slam_gmapping.launch prefix:=summit_xl_a_

You should see something like this in RVIZ (or *Tools/Graphical Tools*):

![Rviz Gmapping 1](images/rdc_rviz_gmapping1.png)

We have used the **ROS_NAMESPACE** environment variable to run everything inside the namespace *summit_xl_a*. This is because the all the launch files are intended to work in a "multirobot ecosystem", so we need to isolate all the nodes and topics in different namespaces.

The argument **prefix** adds a prefix to all the frames of the robot, including map, odom, base_footprint, ...

Below the default config file:

In [ ]:
<?xml version="1.0"?>
<launch>
    <arg name="prefix" default="summit_xl_"/>
    <arg name="scan_topic" default="front_laser/scan"/>
    <arg name="map_frame" default="$(arg prefix)map"/> 
    <arg name="odom_frame" default="$(arg prefix)odom"/> 
    <arg name="base_frame" default="$(arg prefix)base_link"/> 

    <node pkg="gmapping" type="slam_gmapping" name="slam_gmapping">
      <remap from="scan" to ="$(arg scan_topic)"/>
      <param name="map_frame" value="$(arg map_frame)"/>
      <param name="base_frame" value="$(arg base_frame)"/>
      <param name="odom_frame" value="$(arg odom_frame)"/>
      <param name="map_udpate_interval" value="2.0"/>
      <param name="maxUrange" value="16.0"/>
      <param name="sigma" value="0.05"/>
      <param name="kernelSize" value="1"/>
      <param name="lstep" value="0.05"/>
      <param name="astep" value="0.05"/>
      <param name="iterations" value="5"/>
      <param name="lsigma" value="0.075"/>
      <param name="ogain" value="3.0"/>
      <param name="lskip" value="0"/>
      <param name="srr" value="0.1"/>
      <param name="srt" value="0.2"/>
      <param name="str" value="0.1"/>
      <param name="stt" value="0.2"/>
      <param name="linearUpdate" value="0.2"/>
      <param name="angularUpdate" value="0.1"/>
      <param name="temporalUpdate" value="3.0"/>
      <param name="resampleThreshold" value="0.5"/>
      <param name="particles" value="100"/>
      <param name="xmin" value="-50.0"/>
      <param name="ymin" value="-50.0"/>
      <param name="xmax" value="50.0"/>
      <param name="ymax" value="50.0"/>
      <param name="delta" value="0.05"/>
      <param name="llsamplerange" value="0.01"/>
      <param name="llsamplestep" value="0.01"/>
      <param name="lasamplerange" value="0.005"/>
      <param name="lasamplestep" value="0.005"/>
    </node>
</launch>


The gmapping default configuration works pretty well. For further information visit the [gmapping ROS wiki](http://wiki.ros.org/gmapping)

Anyway, the basic parameters and configuration you should check are:

* **scan** topic. Sometimes we don't subscribe correctly if using namespaces or different topic names than 'scan'
* **frames**. It's also important that all the frame names are correct.
* **map_udpate_interval**. We need a value to get a good performance of the algorithm vs the computational load.
* **maxUrange**. Short ranges in wide areas/environments will not create a valid map.
* **linearUpdate,angularUpdate,temporalUpdate**. It establishes how fast the scans are going to be processed while the robot is moving.

### 1.3 Moving the robot through the environment

Once we have the robot and gmapping running we can start moving around with the robot to create the map. To do that we can use several ways. This time we'll use the teleop panel of rviz, that allows publishing linear and angular velocity references. There's a custom version of the standard teleop_panel in the workspace. This version allows setting the speed and disabling the command publication into the cmd_vel topic.

To add the panel we have to:
1. go to "Panels/Add New Panel/teleop_panel/Teleop". 
2. set the command topic "/summit_xl_a/pad_teleop/cmd_vel"
3. interact with the visual marker to set linear and angular speed.

![teleop panel](images/rdc_teleop_panel.png)

As soon as we move and rotate the robot, the new map should start to grow. 

### 1.4 Saving the map

When we consider we have a decent map, we can save into a file.

We will save the map by using the **map_saver** node from map_server package.

We have to open a new terminal an execute:


In [ ]:
ROS_NAMESPACE=summit_xl_a rosrun map_server map_saver -f map1

In [ ]:
[ INFO] [1529429306.186739706]: Waiting for the map
[ INFO] [1529429306.464045430, 417.989000000]: Received a 1984 X 1984 map @ 0.050 m/pix
[ INFO] [1529429306.464131838, 417.989000000]: Writing map occupancy data to map1.pgm
[ INFO] [1529429306.918780244, 418.094000000]: Writing map occupancy data to map1.yaml
[ INFO] [1529429306.918944170, 418.094000000]: Done

The command will save a map with the name "map1", specified by the argument "-f".
If everything is ok, the new map (.pgm and .yaml) should be located in the current path where you run the command. 

Now we are going to copy the map into the package **summit_xl_localization**.


In [ ]:
mkdir ~/simulation_ws/src/summit_xl_common/summit_xl_localization/maps/map1/
mv map1.* ~/simulation_ws/src/summit_xl_common/summit_xl_localization/maps/map1/

Now that we have the map we can stop gmapping by closing the terminal where it was running or pressing "CTRL+C". Once stopped, the transform between summit_xl_a_map and summit_xl_a_odom will dissapear.

A full map is available as **rds_full** in the maps folder.

It's time to move to the next topic! 

## 2. How to use the map to localize the robot

In this section we will use the map created previosly and we will localize the robot on it. To do that we will use the packages [map_server](http://wiki.ros.org/map_server) and [amcl](http://wiki.ros.org/amcl).

 * **amcl** is a probabilistic localization system for a robot moving in 2D. It implements the adaptive (or KLD-sampling) Monte Carlo localization approach (as described by Dieter Fox), which uses a particle filter to track the pose of a robot against a known map.

### 2.1 Running map_server

We will run a map server that will use the map created in the previous section. 

Open a new terminal and execute:

In [ ]:
ROS_NAMESPACE=summit_xl_a roslaunch summit_xl_localization map_server.launch map_file:=rds_full/rds_full.yaml prefix:=summit_xl_a_

Having a look to the launch file...

In [ ]:
<?xml version="1.0"?>
<launch>
    <!-- maps inside the folder summit_xl_navigation/maps -->
    <arg name="map_file" default="map1/map1.yaml"/>
    <arg name="prefix" default="summit_xl_"/>
    <arg name="frame_id" default="$(arg prefix)map"/>

    <!-- Run the map server -->
    <node name="map_server" pkg="map_server" type="map_server" args="$(find summit_xl_localization)/maps/$(arg map_file)">
        <param name="frame_id" value="$(arg frame_id)"/>
    </node>

</launch>

### 2.2 Running amcl

To work with amcl we need:

* 2D Laserscan
* Tfs (transforms)
* A map

Now we will launch the amcl node. Open a new terminal a execute:

In [ ]:
ROS_NAMESPACE=summit_xl_a roslaunch summit_xl_localization amcl.launch prefix:=summit_xl_a_ odom_model_type:=omni

In [ ]:
process[summit_xl_a/amcl-1]: started with pid [19814]
[ INFO] [1529431699.800209420, 1130.846000000]: Requesting the map...
[ INFO] [1529431699.820685002, 1130.848000000]: Received a 1984X 1984 map @ 0.050 m/pix

[ INFO] [1529431700.310701811, 1131.012000000]: Initializing likelihood field model; this can take some time on large maps...
[ INFO] [1529431700.640262545, 1131.110000000]: Done initializing likelihood field model.

Having a look to launch file...

In [ ]:
<?xml version="1.0"?>
<launch>

<arg name="prefix" default="summit_xl_"/>
<arg name="x_init_pose" default="0"/>
<arg name="y_init_pose" default="0"/>
<arg name="z_init_pose" default="0"/>
<arg name="scan_topic" default="front_laser/scan"/>
<arg name="map_topic" default="map"/>
<arg name="global_frame" default="$(arg prefix)map"/>
<arg name="odom_frame" default="$(arg prefix)odom"/>
<arg name="base_frame" default="$(arg prefix)base_footprint"/>
<arg name="odom_model_type" default="diff"/> <!-- omni -->

<node pkg="amcl" type="amcl" name="amcl" output="screen">
  <remap from="scan" to="$(arg scan_topic)"/>
  <remap from="map" to="$(arg map_topic)"/>

  <!-- Publish scans from best pose at a max of 10 Hz -->
  <param name="use_map_topic" value="false"/>
  <param name="odom_model_type" value="$(arg odom_model_type)"/>
  <param name="odom_alpha5" value="0.1"/>
  <param name="transform_tolerance" value="0.2" />
  <param name="gui_publish_rate" value="10.0"/>
  <param name="laser_max_beams" value="30"/>
  <param name="min_particles" value="500"/>
  <param name="max_particles" value="5000"/>
  <param name="kld_err" value="0.05"/>
  <param name="kld_z" value="0.99"/>
  <param name="odom_alpha1" value="0.2"/>
  <param name="odom_alpha2" value="0.2"/>
  <!-- translation std dev, m -->
  <param name="odom_alpha3" value="0.8"/>
  <param name="odom_alpha4" value="0.2"/>
  <param name="laser_z_hit" value="0.5"/>
  <param name="laser_z_short" value="0.05"/>
  <param name="laser_z_max" value="0.05"/>
  <param name="laser_z_rand" value="0.5"/>
  <param name="laser_sigma_hit" value="0.2"/>
  <param name="laser_lambda_short" value="0.1"/>
  <param name="laser_lambda_short" value="0.1"/>
  <param name="laser_model_type" value="likelihood_field"/>
  <param name="laser_likelihood_max_dist" value="2.0"/>
  <param name="update_min_d" value="0.2"/>
  <param name="update_min_a" value="0.5"/>
  <param name="odom_frame_id" value="$(arg odom_frame)"/>
  <param name="base_frame_id" value="$(arg base_frame)"/>
  <param name="global_frame_id" value="$(arg global_frame)"/>
  <param name="resample_interval" value="1"/>
  <param name="transform_tolerance" value="0.1"/>
  <param name="recovery_alpha_slow" value="0.0"/>
  <param name="recovery_alpha_fast" value="0.0"/>

  <param name="initial_pose_x" value ="$(arg x_init_pose)"/>
  <param name="initial_pose_y" value ="$(arg y_init_pose)"/>
  <param name="initial_pose_z" value ="$(arg z_init_pose)"/>
  <param name="initial_pose_a" value ="0.0"/>

</node>

</launch>

After launching amcl, first thing we need to do is to init the pose of the robot in the map. We can do it by publishing into the topic "/summit_xl_a/initialpose" or by the RVIZ tool "2D Pose Estimate", moving the arrow where we consider the robot is located.

![amcl 1](images/rdc_amcl1.png)

When we set the pose, the robot model will move to that location and if we have a PoseArray marker subscribed to the "/summit_xl_a/particlecloud" we can see how the particle filter is working and the quality of the pose estimation.


![amcl 2](images/rdc_amcl2.png)

Moving the robot around, and if the initial position was correct, the particles dispersion will get reduced little by little.

![amcl 3](images/rdc_amcl3.png)

## 3 How to use the map and localization to make the robot move autonomously on the environment while avoiding obstacles


In this section we will use the [move_base](http://wiki.ros.org/move_base) package to navigate autonomously by the current world and using the map we created before as well as localizing the robot with amcl.

 * **move_base** provides an implementation of an action that, given a goal in the world, will attempt to reach it with a mobile base


### 3.1 Running move_base

To work with move_base we need:

* 2D Laserscan or Pointcloud
* Tfs (transforms)
* Odometry source
* A map
* A base controller

![move_base diagram](http://wiki.ros.org/move_base?action=AttachFile&do=get&target=overview_tf.png)

Move base is probably the most complex package that we've been seeing in this course.

To launch move_base, open a new terminal and execute:

In [ ]:
ROS_NAMESPACE=summit_xl_a roslaunch summit_xl_navigation move_base.launch prefix:=summit_xl_a_ omni:=true

Having a look to the launch file...

In [ ]:
<?xml version="1.0"?>
<launch>
  <!-- TEB LOCAL PLANNER -->
  <arg name="prefix" default="summit_xl_"/>
  <arg name="cmd_vel_topic" default="move_base/cmd_vel"/> <!-- using twist mux -->
  <arg name="odom_topic" default="robotnik_base_control/odom"/>
  <arg name="global_frame" default="$(arg prefix)map"/>
  <arg name="odom_frame" default="$(arg prefix)odom"/>
  <arg name="base_frame" default="$(arg prefix)base_footprint"/>
  <arg name="scan_topic" default="front_laser/scan"/>
  <arg name="omni" default="false"/>

  <!-- Run move_base -->
  <node pkg="move_base" type="move_base" respawn="false" name="move_base">
    <rosparam file="$(find summit_xl_navigation)/config/move_base_params.yaml" command="load" />
    <rosparam file="$(find summit_xl_navigation)/config/costmap_common_params.yaml" command="load" ns="global_costmap" />
    <rosparam file="$(find summit_xl_navigation)/config/costmap_common_params.yaml" command="load" ns="local_costmap" />
    <rosparam file="$(find summit_xl_navigation)/config/local_costmap_params.yaml" command="load" />
    <rosparam file="$(find summit_xl_navigation)/config/global_costmap_params_map.yaml" command="load" />
    <rosparam if="$(arg omni)" file="$(find summit_xl_navigation)/config/teb_local_planner_omni_params.yaml" command="load" />
    <rosparam unless="$(arg omni)" file="$(find summit_xl_navigation)/config/teb_local_planner_diff_params.yaml" command="load" />
    <!-- reset frame_id parameters using user input data -->
    <param name="global_costmap/obstacle_layer/scan/sensor_frame" value="$(arg prefix)front_laser_link"/>
    <param name="local_costmap/obstacle_layer/scan/sensor_frame" value="$(arg prefix)front_laser_link"/>
    <param name="global_costmap/obstacle_layer/scan/topic" value="$(arg scan_topic)"/>
    <param name="local_costmap/obstacle_layer/scan/topic" value="$(arg scan_topic)"/>
    <param name="local_costmap/global_frame" value="$(arg odom_frame)"/>
    <param name="local_costmap/robot_base_frame" value="$(arg base_frame)"/>
    <param name="global_costmap/global_frame" value="$(arg global_frame)"/>
    <param name="global_costmap/robot_base_frame" value="$(arg base_frame)"/>
    <param name="DWAPlannerROS/global_frame_id" value="$(arg odom_frame)"/>
    <param name="TebLocalPlannerROS/map_frame" value="$(arg global_frame)"/>
    <param name="base_local_planner" value="teb_local_planner/TebLocalPlannerROS" />
    <param name="controller_frequency" value="5.0" />
    <param name="controller_patience" value="15.0" />
    <remap from="cmd_vel" to="$(arg cmd_vel_topic)" />
    <remap from="odom" to="$(arg odom_topic)" />
  </node>

</launch>

After running move_base we should be able to send goals to the robot by using the action "/summit_xl_a/move_base", the topic "/summit_xl_a/move_base_simple/goal" or the RVIZ tool "2D Nav Goal".

![move_base 1](images/rdc_movebase1.png)

### 3.2 Rviz Navigation Displays

In this section we are going to configure RVIZ to provide really useful information about planning and costmap.

* Global costmap
* Global footprint
* Local costmap
* Local footprint for Teb is not available
* Global plan
* Local plan
* Current goal

After adding all of this visualization displays to RVIZ, it should look like this:

![move_base 2](images/rdc_movebase2.png)

Now, if we add some obstacles in the Gazebo world and set different goals, we'll see how move_base is plannig and avoiding collisions.

### 3.3 move_base config files (optional)

In this section we will have a look to main configuration files and parameters for move_base.

Some of this params are overwritten in the launch file, above all all of them related with frames.

#### move_base_params.yaml

It contains global params.

Examples:

* **controller_frequency**: The rate in Hz at which to run the control loop and send velocity commands to the base.
* **controller_patience**: How long the controller will wait in seconds without receiving a valid control before space-clearing operations are performed.

* **planner_frequency**: The rate in Hz at which to run the global planning loop. If the frequency is set to 0.0, the global planner will only run when a new goal is received or the local planner reports that its path is blocked
* **planner_patience**: How long the planner will wait in seconds in an attempt to find a valid plan before space-clearing operations are performed.


#### costmap_common_params.yaml

Costmap params common for global and local costmap.

Examples:


* footprint: Specification for the footprint of the robot.

* inflation_layer:
 * inflation_radius: The radius in meters to which the map inflates obstacle cost values.

* obstacle_layer:
 * obstacle_range: The default maximum distance from the robot at which an obstacle will be inserted into the cost map in meters. This can be over-ridden on a per-sensor basis.
 * raytrace_range: the default range in meters at which to raytrace out obstacles from the map using sensor data. This can be over-ridden on a per-sensor basis.
 * observation_sources: defines a namespace in which parameters can be set
 * scan: {sensor_frame: summit_xl_front_laser_link, data_type: LaserScan, topic: scan, marking: true, clearing: true}
    

#### local_costmap_params.yaml

Costmap params specific for local costmap

#### global_costmap_params_map.yaml

Costmap params specific for global costmap

